In [ ]:
%pip install opendatasets as 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


import opendatasets as od
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
od.download("https://www.kaggle.com/competitions/goodreads-books-reviews-290312/data?select=goodreads_test.csv")


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading goodreads-books-reviews-290312.zip to ./goodreads-books-reviews-290312


100%|██████████| 635M/635M [01:11<00:00, 9.24MB/s] 



Extracting archive ./goodreads-books-reviews-290312/goodreads-books-reviews-290312.zip to ./goodreads-books-reviews-290312


FileNotFoundError: [Errno 2] No such file or directory: '../input/goodreads-books-reviews-290312/goodreads_test.csv'

In [7]:
df = pd.read_csv("./goodreads-books-reviews-290312/goodreads_test.csv")
print(df.head())
print(df.size)


4780330